# Empirical IO PhD Class
## Problem Set 0
Jimena, Eyal and Pietro 

September 2020

## Problem 0: Logit Function
** 1. The log-sum-exp function is convex everywhere: **

Pick any distinct $x, y \in \mathbb{R}^{N+1}$ and any $\alpha \in (0,1)$. 
$$f ( \alpha x + ( 1 - \alpha ) y ) = \log \sum_{i=0}^N \exp(\alpha x_i + ( 1 - \alpha ) y_i)$$
Applying Hölder's inequality to $\sum_{i=0}^N \exp(\alpha x_i) \exp( ( 1 - \alpha ) y_i)$ with exponents $\frac{1}{\alpha}$ and $\frac{1}{1-\alpha}$ we get 
$$ \sum_{i=0}^N \exp(\alpha x_i) \exp( ( 1 - \alpha ) y_i ) \leq \left [ \sum_{i=0}^N |\exp(\alpha x_i)|^{\frac{1}{\alpha}}\right ]^\alpha \left [ \sum_{i=0}^N |\exp((1 - \alpha) y_i)|^{\frac{1}{1 - \alpha}} \right ]^{1 - \alpha}$$
Taking logs on both sides and rearranging: 
$$ \log \sum_{i=0}^N exp( \alpha x_i + ( 1 - \alpha ) y_i) \leq \alpha \log \sum_{i=0}^N exp( x_i ) + ( 1 - \alpha )\log \sum_{i=0}^N exp( y_i )  $$
So the function is convex everywhere. 

** 2. Using the max trick: **

Fix some $x \in \mathbb{R}^{N+1}$ and let $m:=\max_i x_i$. Assume wlog $x_0=m$. 
We have 
$$ IV = \log \sum_{i=0}^N exp(x_i) = \log \sum_{i=0}^N exp(x_i) \frac{exp(m)}{exp(m)}  $$
and rearranging 
$$ IV = m + \log ( 1 + \sum_{i=1}^N \exp ( x_i-m ) ) $$
We have rescaled everything relative to the $\max$ and added a constant. With this we take the exponential of smaller numbers and avoid the overflow problem. 

** 3. Comparing it to scipy.misc.logsumexp. Does it appear to suffer from underflow/overflow?
Does it use the max trick?**

First generate a tuple of values which includes some $x_i>600$ and evaluate the function at this point.

In [117]:
import numpy as np
from scipy.special import logsumexp

x=np.arange(10, 800, 10)


If we calculate the original IV equation we get the error (the evaluated number is infinity):

In [118]:
IV_1=np.log(np.sum(np.exp(x)))
IV_1

inf

Now we do the *max* trick and compare to the value that we get from the logsumexp function:

In [119]:
m=max(x)
IV=m+np.log(1+np.sum(np.exp(x-m)))
IV


790.6931698812978

In [120]:
logsumexp(x)

790.0000454009604

The logsumexp and the function we modified have similar results but not exactly the same so logsumexp is doing something else to compute IV.

## Problem 1: Stationary Distribution from eigenvectors
Write a function that computes the ergodic distribution of the matrix
$P$ by examining the properly rescaled eigenvectors and compare your result to $P^{100}$
We first define the transition matrix P and take the 100th power of it.

In [121]:
P = np.array([[0.2, 0.4, 0.4],[0.1, 0.3, 0.6],[0.5, 0.1, 0.4]])
P_100 = np.linalg.matrix_power(P, 100)
P_100

array([[0.31034483, 0.24137931, 0.44827586],
       [0.31034483, 0.24137931, 0.44827586],
       [0.31034483, 0.24137931, 0.44827586]])

Now to calculate the stationary distribution we calculate the eigenvalues and left eigenvectors of P. 

In [153]:
P_T = np.matrix.transpose(P)
w,v = np.linalg.eig(P_T)
w

array([ 1.  +0.j        , -0.05+0.23979158j, -0.05-0.23979158j])

In [154]:
v

array([[-0.52048344+0.j        ,  0.66666667+0.j        ,
         0.66666667-0.j        ],
       [-0.40482045+0.j        , -0.41666667-0.39965263j,
        -0.41666667+0.39965263j],
       [-0.75180941+0.j        , -0.25      +0.39965263j,
        -0.25      -0.39965263j]])

We take the unit eigenavlue and rescale the corresponding left eigenvectos

In [155]:
d = np.sum(np.real(v[0]))
pi = v[0].real/d
pi

array([-0.64031925,  0.82015963,  0.82015963])

In [152]:
val, vec = np.linalg.eig(P)
print(P.dot(np.matrix.transpose(vec[0])))
print(val[0]*vec[0])

[-0.06962997+0.j         -0.00616493+0.15310378j -0.26002508+0.15310378j]
[-0.57735027+0.j          0.05730011-0.51034592j  0.05730011+0.51034592j]


## Problem 1 Stationaty Distribution from system of equations
That approach is not working out for some reason so I solved it in a different way below and I got a pretty good match to $P^{100}$

We want to have $\pi$ such that $ (P^T-I)\pi=0 $ and $\sum_i \pi_i=1$. We can write this as a system of equations $A\pi=b$ with $A^T=[P^T-I, \mathbb{1}]$ and $b=[\mathbb{0}, 1]^T$ so that we can solve for $\pi$ in $A^T A \pi=A^T b$

In [124]:
def stat_distr(M):
    A = np.append(np.matrix.transpose(M) - np.identity(len(M)), [np.ones(len(M))], axis=0)
    A_T = np.matrix.transpose(A)
    b = np.matrix.transpose( np.append( [ np.zeros( len(M)) ], [1] ) )
    x = np.linalg.solve( A_T.dot(A), A_T.dot(b) )
    return x

In [125]:
stat_distr(P)

array([0.31034483, 0.24137931, 0.44827586])

In [156]:
P_100

array([[0.31034483, 0.24137931, 0.44827586],
       [0.31034483, 0.24137931, 0.44827586],
       [0.31034483, 0.24137931, 0.44827586]])

## Problem 2: Numerical Integration

In [179]:
import scipy as sp
from scipy import stats
def binomiallogit(b, pdf=sp.stats.norm.pdf):
    return (np.exp(b*.5)/(1+np.exp(b*.5)))*pdf(b, 0.5, np.sqrt(2))
    

    

In [206]:
a= binomiallogit(-100)
print(a)

0.0


In [211]:
sp.integrate.quad(binomiallogit, -10000, 10000, epsrel=10^(-14))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\scipy\integrate\quadpack.py:385: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)


(nan, nan)